In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
df = pd.read_csv('/workspaces/proyectfinaltour/data/processed/unificado/weather.csv')

In [6]:
scaler = StandardScaler()
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

In [7]:
X = df[['maxtempC', 'mintempC', 'tempC', 'humidity', 'windspeedKmph']]
y = df['uvIndex']  # Ejemplo: predecir el índice UV

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

In [10]:
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

In [11]:
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

In [12]:
results = {
    'Model': ['Linear Regression', 'Random Forest', 'Gradient Boosting'],
    'MSE': [mse_lr, mse_rf, mse_gb],
    'R2 Score': [r2_lr, r2_rf, r2_gb]
}

results_df = pd.DataFrame(results)
results_df

,Model,MSE,R2 Score
0,Linear Regression,0.095193,0.904328
1,Random Forest,0.077387,0.922224
2,Gradient Boosting,0.074537,0.925087


-1. Regresión Lineal
MSE: 1.121
R2 Score: 0.856
La Regresión Lineal muestra un error cuadrático medio (MSE) de 1.121 y un coeficiente de determinación (R2 Score) de 0.856. Esto indica que el modelo captura aproximadamente el 85.6% de la variabilidad en el índice UV utilizando las características climáticas proporcionadas. Aunque el modelo proporciona una buena estimación, no es tan preciso como los otros modelos.

-2. Random Forest
MSE: 0.273
R2 Score: 0.970
El modelo Random Forest presenta un MSE de 0.273 y un R2 Score de 0.970. Este modelo es significativamente más preciso que la Regresión Lineal, capturando el 97% de la variabilidad en los datos. La capacidad de los Random Forest para manejar relaciones no lineales y su robustez frente al sobreajuste lo convierten en una opción excelente para este conjunto de datos.

-3. Gradient Boosting
MSE: 0.291
R2 Score: 0.967
El modelo de Gradient Boosting muestra un MSE de 0.291 y un R2 Score de 0.967. Similar al Random Forest, Gradient Boosting es muy preciso y captura el 96.7% de la variabilidad en los datos. Este modelo es efectivo en la mejora de la precisión a través de la combinación de múltiples modelos débiles.

# Interpretación
Los resultados indican que tanto el modelo de Random Forest como el de Gradient Boosting superan significativamente a la Regresión Lineal en términos de precisión y capacidad predictiva. Esto se debe a la capacidad de estos modelos de captura de relaciones no lineales y de manejo de la complejidad del dataset. El Random Forest ligeramente supera al Gradient Boosting en este caso específico, aunque ambos modelos ofrecen una precisión muy alta.